<u>IMPORTS</u>

In [60]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import astropy.constants as c 
import astropy.units as u

<u>CONSTANTS</u>

In [62]:
KB = c.k_B.cgs.value            #Boltzmann Constant
PI = np.pi                      #PI
E_B = (4.52 * u.eV).cgs.value   #Decorrelation Energy
H = c.h.cgs.value               #Planck's Constant
HBAR = H/(2*PI)                 #h-bar
W_VIB = 5.4e14                  #vibrational frequency for molecular hydrogen
MOI = 9.1e-41                   #moment of inertia for molecular hydrogen
I = (13.6 * u.eV).cgs.value     #ionization energy
A0 = c.a0.cgs.value             #bohr radius
M_H = 1.67e-24                  #mass of hydrogen in grams
M_P = c.m_p.cgs.value           #mass of proton in grams
M_E = c.m_e.cgs.value           #mass of electron in grams

index = np.arange(0, 101, 1)    #for quantum numbers n and angular momentum numbers j (0-100)

<u>PARTITION FUNCTIONS</u>

In [64]:
#H2 internal partition function

#Because H2 is molecular hydrogen, we must take into account the electronic, vibrational, 
#and rotational partition functions when computing the internal partition functionn

'''
H2 electronic partition function

Params
T : temperature

Returns
H2 electronic partition function at temp T
'''
def h2_elec(T):
    return 1 + np.exp(-E_B/(KB * T))


'''
H2 rotation partition function at angular momentum number

Params
T: temperature
j: angular momentum number

Returns
H2 rotational partition function at temp T and angular momentum num j
'''
def rot_deg(T, j):
    return (2*j + 1) * np.exp(-(j * (j + 1) * HBAR ** 2 / (2 * MOI)) / (KB * T))


'''
H2 vibrational partition function quantum number

Params
T: temperature
n: quantum number

Returns
H2 vibrational partition function at temp T and quantum num n
'''
def vib_f(T, n):
    return np.exp(-(n + 0.5) * HBAR * W_VIB / (KB * T))


'''
H2 partition functions summation over all quantum and angular momentum numbers

Params:
T : temperature

Returns:
H2 vibrational partition function over all quantum numbers n
H2 rotational partition function for all angular momentum numbers j (para + ortho)
'''
def parallel(Temp):
    z_vib = np.sum(vib_f(T, index))
    z_para = np.sum(rot_deg(T, index[0:100:2]))
    z_ortho = np.sum(3*rot_deg(T, index[1:100:2]))
            
    return z_vib, z_para + z_ortho


'''
H2 internal partition function

Params:
T : temperature

Returns:
H2 internal partition function: z_h2_int = z_elec * z_rot * z_vib
'''
def h2_int(T):
    np.mean(T)
    z_elec = h2_elec(T)
    z_vib, z_rot = parallel(T)
    
    return z_elec * z_rot * z_vib

In [65]:
#HI internal partition function

#Only consists of the HI electronic partition function because it is monatomic 
#hydrogen--no vibrational or rotational partition functions 

'''
Calculating n_max for upperbound on summation

Params:
T : temperature
P : pressure

Returns:
n_max : maximum energy level given by balancing the effective radius of electron 
        shell n^2 * a0 (n is our n_max and a0 is the bohr radius) with the (# density)^(-1/3)/2
        to avoid e- state overlap (Lecture 17)
        i.e. n_max^2 * a0 = (# density)^(-1/3)/2
'''
def n_max(T, P):
    return np.sqrt((KB * T) ** (1/3) / (2 * A0 * P ** (1/3)))


'''
HI electronic partition function per energy level n

Params:
T : temperature
n : energy level

Returns:
HI electronic partition function at temperature T and energy level n
'''
def h1_exp(T, n):
    return n ** 2 * np.exp(-I * (1 - (1/(n**2)))/(KB * T))


'''
HI electronic partition function

Params:
T : temperature
P : Pressure

Returns:
HI electronic partition function at temp T and pressure P summed over all possible energy levels (1 to n_max)
'''
def h1_elec(T, P):
    n_m = n_max(T, P)
    n_elec = np.arange(1, int(n_m)+1, 1)

    z_sum = np.sum(h1_exp(T, n_elec))

    return 4 * z_sum
            

<u>IONIZATION FRACTIONS</u>

In [67]:
#Function for Q2.1:
#computing ionization fraction for HI - x_HII

'''
RHS of ionization fraction for HI expression derived in question 2 part 1

Params:
T : temperature
P : pressure

Returns:
RHS of ionization fraction Q2.1 derived for a temperature T and pressure P
'''
def C(T, P):
    z_tot = h1_elec(T, P) ** 2 / h2_int(T)
    
    return (T * KB / P) * (PI * KB * T/ H ** 2) ** (3/2) * (M_H ** (3/2)) * z_tot * np.exp(-E_B / (KB * T))


'''
Ionization Fraction x_HI = n_HI/n_tot

Because our LHS of the expression derived in Q2.1 is x^2/(1-x), when expanded with RHS, 
we need to compute the roots of the quadratic with respect to x. This function is the
computation of that quadratic with C(T, P) as the RHS of expression from Q2.1 as stated 
in the above function

Params:
T : temperature
P : pressure

Returns:
x_HI : HI ionization fraction at a specific temperature T and pressure P
'''
def ion_frac(T, P):

    return (-C(T,P) + np.sqrt(C(T,P) ** 2 + (4 * C(T,P))))/2

In [68]:
#Function for Q2.3:
#computing ionization fraction for HII - x_HII

'''
RHS of ionization fraction for HII expression derived in question 2 part 3 using the SAHA equation

Params:
T : temperature
P : pressure

Returns:
RHS of ionization fraction Q2.3 derived for a temperature T and pressure P
'''
def ChII(T, P):
    return (KB * T / P) * (2 * PI * KB * T / H ** 2) ** (3/2) * (M_P * M_E / M_H) ** (3/2) * (4 / h1_elec(T, P)) * np.exp(-I / (KB * T))


'''
Ionization Fraction x_HII = n_HII/n_I

Because our LHS of the expression derived in Q2.3 is x^2/(1-x), when expanded with RHS, 
we need to compute the roots of the quadratic with respect to x. This function is the
computation of that quadratic with ChII(T, P) as the RHS of expression from the SAHA 
equation as stated in the above function

Params:
T : temperature
P : pressure

Returns:
x_HII : HII ionization fraction at a specific temperature T and pressure P
'''
def ion_hII(T, P):
    
    return (-ChII(T,P) + np.sqrt(ChII(T,P) ** 2 + (4 * ChII(T,P))))/2

<u>SPECIFIC HELMHOLTZ FREE ENERGIES AND SPECIFIC ENTROPY OF MIXING</u>

We have that our helmholtz free energy is $F = - k_bT \ln{(Z)}$ where Z is our partition func, T temp, and $k_b$ boltzmann constant. To get 
<i><b>specific helmholtz free energy</b></i>
, we must divide by mass of constituent particle of species i: $\frac{F}{N_im_i}$

Last cell is for specific entropy of mixing!

In [70]:
#H2 specific helmholtz free energy


'''
Specific helmholtz free energy for H2

Params:
P : pressure
xhI : HI ionization fraction for a specific temp T and P

Returns:
F : Specific helmholtz free energy for H2 at a fixed pressure P as a function of temperature T
'''
def f_h2_noconst(P, xhI):

    '''
    Wrapper function for helmholtz free energy for H2

    We need this in order to take derivative of our specific helmholtz free energy with respect
    to T to get specific entropy S for H2
    
    Params:
    T : temperature
    
    Returns:
    F : Specific helmholtz free energy for H2 as a function of T
    '''
    def F(T):
        xh2 = (1-xhI) #to get x_H2, we take a look at the equation from Q2.1 where n_tot = n_HI + n_HII, 
                      #divide that by n_tot, and use it to write x_H2 = 1 - x_HI
        
        z = h2_int(T)
        return KB * T / (2 * M_H) * np.log((KB * T) / (P * xh2) * (4 * PI * M_H * KB * T / H ** 2) ** (3/2) * z)
        
    return F

In [71]:
#HI specific helmholtz free energy

'''
Specific helmholtz free energy for HI

Params:
P : pressure
xhI : HI ionization fraction for a specific temp T and P

Returns:
F : Specific helmholtz free energy for HI at a fixed pressure P as a function of temperature T
'''
def f_hI_noconst(P, xhI):

    '''
    Wrapper function for helmholtz free energy for HI

    We need this in order to take derivative of our specific helmholtz free energy with respect
    to T to get specific entropy S for HI
    
    Params:
    T : temperature
    
    Returns:
    F : Specific helmholtz free energy for HI as a function of T
    '''
    def F(T):
        z = h1_elec(T, P)
        return T * KB / M_H * np.log((KB * T) / (P * xhI) * (2 * PI * M_H * KB * T / H ** 2) ** (3/2) * z)
        
    return F


In [72]:
#HII specific helmholtz free energy

'''
Specific helmholtz free energy for HII

Params:
P : pressure
xhII : HII ionization fraction for a specific temp T and P

Returns:
F : Specific helmholtz free energy for HII at a fixed pressure P as a function of temperature T
'''
def f_hII_noconst(P, xhII):

    '''
    Wrapper function for helmholtz free energy for HII

    We need this in order to take derivative of our specific helmholtz free energy with respect
    to T to get specific entropy S for HII
    
    Params:
    T : temperature
    
    Returns:
    F : Specific helmholtz free energy for HII as a function of T
    '''
    def F(T):
        return T * KB / M_P * np.log(KB * 2 * T / (P * xhII) * (2 * PI * M_P * KB * T / H ** 2) ** (3/2))
        
    return F

In [73]:
#Specific entropy of mixing

'''
Specific helmholtz free energy for HII

Params:
P : pressure
xhII : HII ionization fraction for a specific temp T and P

Returns:
F : Specific helmholtz free energy for HII at a fixed pressure P as a function of temperature T
'''
def inLN(X, mu):
    return X / (mu * M_H)

'''
Specific entropy of mixing

Params:
xh1 : HI ionization fraction for a specific temp T and P
xhII : HII ionization fraction for a specific temp T and P

Returns:
S_mix : Specific entropy of mixing
'''
def s_mix(xh1, xh11):
    x2 = (1-xh1) #to get x_H2, we take a look at the equation from Q2.1 where n_tot = n_HI + n_HII, 
                 #divide that by n_tot, and use it to write x_H2 = 1 - x_HI

    #masses of each constituent particle and the total mass
    mass_h2 = x2*2*M_H
    mass_hI = xh1*M_H
    mass_hII = xh11*M_P
    mass_tot = mass_h2 + mass_hI #similar reasoning as calculating for x_H2 from above for 
                                 #consistency

    #mass fractions per constituent particle
    X_h2 = mass_h2/mass_tot
    X_hI = mass_hI/mass_tot
    X_hII = mass_hII/mass_tot
    
    mass_frac = np.array([X_h2, X_hI, X_hII])
    mu_sum = np.array([2, 1, 1])
    
    return KB * (np.sum(inLN(mass_frac, mu_sum)) * np.log(np.sum(inLN(mass_frac, mu_sum))) - np.sum(inLN(mass_frac, mu_sum) * np.log(inLN(mass_frac, mu_sum))))